In [ ]:
import set_path

import numpy as np
import laspy
import geopandas as gpd

import open3d as o3d

from upcp.utils import ahn_utils, clip_utils
from upcp.utils.interpolation import FastGridInterpolator

from gvl.helper_functions import color_clusters
from gvl.tree_detector import DetectorTree
from gvl.ahn_utils import GeoTIFFReader2

In [ ]:
import warnings  # temporary, to supress deprecationwarnings from shapely
warnings.filterwarnings('ignore')

In [ ]:
RD_CRS = 'epsg:28992'
LL_CRS = 'WGS84'

# AHN classification
AHN_OTHER = 1
AHN_GROUND = 2
AHN_BUILDING = 6
AHN_WATER = 9
AHN_ARTIFACT = 26

In [ ]:
# # Use CycloMedia style tilecodes
# tilecode = '2463_9714'

# las_file = f'../datasets/ahn_{tilecode}.laz'
# ahn_npz_folder = '../../../datasets/Accessibility_oost/ahn/ahn4_npz/'

# ahn_reader = ahn_utils.NPZReader(ahn_npz_folder, caching=False)

In [ ]:
# Use AHN subtiles
tilecode = '25GN1_04'

las_file = f'../datasets/ahn_laz/{tilecode}.LAZ'
ahn_geotiff_folder = '../datasets/ahn_dtm/'

ahn_reader = GeoTIFFReader2(ahn_geotiff_folder, fill_gaps=False,
                            smoothen=True, smooth_thickness=2)

In [ ]:
target_area = gpd.read_file(f'../datasets/validation/correct_trees_areas_oosterpark.shp', crs=RD_CRS)

bbox = target_area.unary_union.bounds

In [ ]:
las = laspy.read(las_file)
pts = np.vstack((las.x, las.y, las.z)).T

### Filter 1: scalar fields and clip
- classification -> 'overig label' points
- number of returns -> ignore nr. 1

In [ ]:
area_mask = clip_utils.rectangle_clip(pts, bbox)
class_mask = las.classification == AHN_OTHER
#returns_mask = las.number_of_returns != 2 # TODO wordt gebruikt in de python file

mask = class_mask & area_mask # & returns_mask

In [ ]:
points_dtype = np.dtype([('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'),
                         ('NZ', '<f8'), ('HAG', '<f8'),
                         ('Classification', '<i4'), ('NumberOfReturns', '<i4'), ('ReturnNumber', '<i4')])
points = np.empty(shape=(np.count_nonzero(mask),), dtype=points_dtype)

In [ ]:
o3d_pc = o3d.geometry.PointCloud()
o3d_pc.points = o3d.utility.Vector3dVector(pts[mask])
o3d_pc.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,
                                                      max_nn=30))

In [ ]:
gnd_tile = ahn_reader.filter_area(bbox)

fast_z = FastGridInterpolator(
            gnd_tile['x'], gnd_tile['y'], gnd_tile['ground_surface'])
gnd_z = fast_z(pts[mask])

In [ ]:
np.count_nonzero(gnd_z == np.nan)

In [ ]:
points['X'] = pts[mask,0]
points['Y'] = pts[mask,1]
points['Z'] = pts[mask,2]
points['NZ'] = np.asarray(o3d_pc.normals)[:,2]
points['HAG'] = pts[mask,2] - gnd_z
points['Classification'] = las.classification[mask] # TODO always AHN_OTHER
points['NumberOfReturns'] = las.number_of_returns[mask]
points['ReturnNumber'] = las.return_number[mask]

### Filter 2: points close to ground
TODO move this before the KDTree code

In [ ]:
above_ground_in_meters = 2
raw_points = points[points['HAG'] >= above_ground_in_meters]

### Filter 3: Noise filter
TODO create

In [ ]:
tree = DetectorTree(raw_points)

In [ ]:
# first clustering step
# tree.hdbscan_on_points(min_cluster_size=30, min_samples=10, xyz=False)
tree.hdbscan_on_points(min_cluster_size=50, min_samples=10, xyz=False)
tree.convex_hullify(points=tree.clustered_points)
# df_to_pg(tree.tree_df, schema='bomen', table_name='xy_bomen')

In [ ]:
tree.tree_df.plot(column='xy_clusterID', cmap='tab20')

In [ ]:
# second cluster step
tree.find_points_in_polygons(tree.tree_df)
# tree.kmean_cluster(tree.xy_grouped_points, round_val=2, min_dist=1)
tree.kmean_cluster(tree.xy_grouped_points, round_val=1, min_dist=3)
tree.convex_hullify(tree.kmean_grouped_points, kmean_pols=True)
# df_to_pg(tree.tree_df, schema='bomen', table_name='km_bomen')

In [ ]:
tree.tree_df.plot(column='xy_clusterID', cmap='tab20')

In [ ]:
tree.tree_df

In [ ]:
data = tree.kmean_grouped_points.to_records()

In [ ]:
# Shuffle classification for better visualisation
labels = np.unique(data['Classification'])
new_labels = labels.copy()

rng = np.random.default_rng()
rng.shuffle(new_labels)

shuffle_dict = {l: n for (l, n) in zip(labels, new_labels)}

shuffle_labels = np.array([shuffle_dict[l] for l in data['Classification']])

In [ ]:
header = laspy.LasHeader(point_format=3, version="1.2")
header.offsets = las.header.offsets
header.scales = las.header.scales

new_las = laspy.LasData(header)

new_las.x = data['X']
new_las.y = data['Y']
new_las.z = data['Z']

new_las.add_extra_dim(laspy.ExtraBytesParams(name="label", type="uint16",
                                         description="Label"))
new_las.label = shuffle_labels

In [ ]:
new_las.write(f'trees_{tilecode}_2.laz')